In [8]:
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

BODY_PARTS = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "LHip": 10, "LKnee": 11, "Chest": 12}

POSE_PAIRS = [["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
              ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
              ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
              ["Chest", "LHip"], ["LHip", "LKnee"]]

# Load the data
input_data = pd.read_csv('dataset/movement.csv')
output_data = pd.read_csv('dataset/animate.csv')

# Split the data into training and testing sets (70% training, 30% testing)
input_train, input_test, output_train, output_test = train_test_split(input_data, output_data, test_size=0.3,
                                                                      random_state=42)

# Create a Random Forest Regressor with MultiOutputRegressor
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
model = MultiOutputRegressor(random_forest)

# Fit the model
model.fit(input_train, output_train)

# Make predictions on the test set
predictions = model.predict(input_test)

# Create an empty canvas
canvas = 255 * np.ones((480, 640, 3), dtype=np.uint8)

# Scale factor for drawing points on the canvas
scale_factor = 10

# Draw points and lines on the canvas
for pose_pair in POSE_PAIRS:
    part1_index = BODY_PARTS[pose_pair[0]]
    part2_index = BODY_PARTS[pose_pair[1]]

    for i in range(len(predictions)):
        # Extract x, y positions from predictions
        x1, y1 = int(predictions[i][2 * part1_index] * scale_factor), int(
            predictions[i][2 * part1_index + 1] * scale_factor)
        x2, y2 = int(predictions[i][2 * part2_index] * scale_factor), int(
            predictions[i][2 * part2_index + 1] * scale_factor)

        # Draw points
        cv2.circle(canvas, (x1, y1), 5, (0, 0, 255), -1)
        cv2.circle(canvas, (x2, y2), 5, (0, 0, 255), -1)

        # Draw lines
        cv2.line(canvas, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display the canvas
cv2.imshow('Pose Estimation', canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
import pandas as pd

# Load the data from 'dataset/pose-record.csv'
pose_data = pd.read_csv('dataset/pose-record.csv')

# Get two values, 10 indices away
index_1 = 5  # Choose your desired index
index_2 = index_1 + 10

# Extract the values at the chosen indices
values_1 = pose_data.iloc[index_1, 1:].values
values_2 = pose_data.iloc[index_2, 1:].values

# Calculate the differences between values
differences = values_2 - values_1

# Print the results
print(f"Values at index {index_1}: {values_1}")
print(f"Values at index {index_2}: {values_2}")
print(f"Differences between values: {differences}")

Values at index 5: [288.  96. 304. 176. 256. 192. 224. 256. 208. 208. 336. 192. 368. 256.
 384. 208. 256. 320. 256. 432. 320. 320. 320. 432. 288. 256.]
Values at index 15: [288.  96. 288. 176. 256. 176. 192. 224. 192. 176. 336. 192. 384. 224.
 384. 224. 256. 320. 240. 432. 320. 320. 320. 432. 288. 256.]
Differences between values: [  0.   0. -16.   0.   0. -16. -32. -32. -16. -32.   0.   0.  16. -32.
   0.  16.   0.   0. -16.   0.   0.   0.   0.   0.   0.   0.]


In [31]:
import pandas as pd
import cv2
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

BODY_PARTS = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "LHip": 10, "LKnee": 11, "Chest": 12}

POSE_PAIRS = [["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
              ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
              ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
              ["Chest", "LHip"], ["LHip", "LKnee"]]

# Load the trained model
trained_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
trained_model.fit(input_train, output_train)

# Get two values, 10 indices away
index_1 = 5  # Choose your desired index
index_2 = index_1 + 10

# Extract the values at the chosen indices, including the "time" feature
values_1 = pose_data.iloc[index_1, :].values
values_2 = pose_data.iloc[index_2, :].values

# Calculate the differences between values
differences = values_2 - values_1

# Reshape the differences for prediction
differences_for_prediction = differences.reshape(1, -1)

# Make predictions using the trained regressor
predicted_output = trained_model.predict(differences_for_prediction)

# Print the results
print(f"Values at index {index_1}: {values_1}")
print(f"Values at index {index_2}: {values_2}")
print(f"Differences between values: {differences}")
print(f"Predicted output: {predicted_output}")

# Visualize the predicted output
canvas = 255 * np.ones((480, 640, 3), dtype=np.uint8)

# Scale factor for drawing points on the canvas
scale_factor = 10

# Draw points and lines for predicted output (purple)
for part, index in BODY_PARTS.items():
    x, y = int(predicted_output[0, 2 * index] * scale_factor), int(predicted_output[0, 2 * index + 1] * scale_factor)
    cv2.circle(canvas, (x, y), 5, (128, 0, 128), -1)

# Draw lines for predicted output (purple)
for pose_pair in POSE_PAIRS:
    part1_index = BODY_PARTS[pose_pair[0]]
    part2_index = BODY_PARTS[pose_pair[1]]

    x1, y1 = int(predicted_output[0, 2 * part1_index] * scale_factor), int(predicted_output[0, 2 * part1_index + 1] * scale_factor)
    x2, y2 = int(predicted_output[0, 2 * part2_index] * scale_factor), int(predicted_output[0, 2 * part2_index + 1] * scale_factor)

    cv2.line(canvas, (x1, y1), (x2, y2), (128, 0, 128), 2)

# Display the canvas
cv2.imshow('Predicted Pose Estimation', canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


C:\Users\yunho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\yunho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\yunho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\yunho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\yunho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid fea

Values at index 5: [  0.6 288.   96.  304.  176.  256.  192.  224.  256.  208.  208.  336.
 192.  368.  256.  384.  208.  256.  320.  256.  432.  320.  320.  320.
 432.  288.  256. ]
Values at index 15: [  1.6 288.   96.  288.  176.  256.  176.  192.  224.  192.  176.  336.
 192.  384.  224.  384.  224.  256.  320.  240.  432.  320.  320.  320.
 432.  288.  256. ]
Differences between values: [  1.   0.   0. -16.   0.   0. -16. -32. -32. -16. -32.   0.   0.  16.
 -32.   0.  16.   0.   0. -16.   0.   0.   0.   0.   0.   0.   0.]
Predicted output: [[  0.483   6.72    0.     -2.88   -0.96   -2.24    0.64   -3.68   -0.8
   -2.88    5.28    8.32    0.16   12.32   -4.8     8.32  -18.88    9.76
    0.     -1.12   -0.16   -2.4     0.      0.16    0.      8.8     0.   ]]
